In [1]:
import numpy as np
from scipy import spatial

In [2]:
class Molecule():
    def __init__(self, elements, coords):
        self.elements = elements
        self.coords = coords
    def get_elements(self):
        return self.elements
    def get_coords(self):
        return self.coords

In [3]:
def get_next_molecule(c, e, d, threshold=2.0):
    # Choose molecule belonging to index 0
    atoms_in = [0]
    while True:
        #print(atoms_in)
        length_in = len(atoms_in)
        atoms_out = atoms_in.copy()
        for i in atoms_in:
            t = d[i,:]
            indices = np.nonzero(t <= threshold)
            atoms_out.extend(indices[0])
        atoms_out = list(set(atoms_out))#; print(atoms_out)
        length_out = len(atoms_out)
        if length_in == length_out:
            break
        else:
            atoms_in = atoms_out
    # Now have indices in current frame, extract data from structures
    d_out = np.delete(d, atoms_out, axis = 0); d_out = np.delete(d_out, atoms_out, axis = 1)
    c_out = np.delete(c, atoms_out, axis = 0)
    mol_out = Molecule([e[i] for i in atoms_out], [c[i] for i in atoms_out])
    e_out = e
    for i in sorted(atoms_out, reverse=True):
        e_out.pop(i)
    return(mol_out, c_out, e_out, d_out)

In [4]:
file = open('TMS-Tet.gjf', 'r')
data = file.readlines()
file.close()

In [5]:
print(data[-10:-4])

[' 1848\n', ' 1849\n', ' 1850\n', ' 1851\n', ' 1852\n', ' 1853\n']


In [6]:
# The following assumes that the GJF file has coordinates starting on line 7 (6 when 0-indexed),
#    and there is a blank line between Cartesians and Connectivity data
coord_start_line= 6
coords = []; atoms = []; i = coord_start_line; test = 'placeholder'
while True:
    test = data[i]
    if test == '\n': break
    t = test.strip().split()
    atoms.append(t[0])
    coords.append(t[1:])
    i += 1
a = np.array(coords, np.float)
dm = spatial.distance_matrix(a, a)

In [7]:
distance_threshold = 2.0
molecules = []
coords = a; elements = atoms; distances = dm
while distances.shape[0] > 0:
    molecule, coords, elements, distances = get_next_molecule(coords, elements, distances, distance_threshold)
    molecules.append(molecule)
    #print(len(molecules))

In [8]:
full_molecule_numatoms = max([len(i.get_elements()) for i in molecules])
num_full_molecules = [len(i.get_elements()) for i in molecules].count(full_molecule_numatoms)
print("Largest atomic count for molecular fragment is {}".format(full_molecule_numatoms))
print("Number of such fragments is {}".format(num_full_molecules))

Largest atomic count for molecular fragment is 58
Number of such fragments is 10


In [9]:
def write_xyz(fn, numatoms, nummols, molecules):
    file = open(fn + '.xyz', 'w')
    file.write("{}\n".format(str(numatoms*nummols)))
    file.write("Test of fragment output for threshold {:3.1f}\n".format(distance_threshold))
    for i in molecules:
        if len(i.get_elements()) == numatoms:
            t1 = i.get_elements()
            t2 = i.get_coords()
            for j in range(numatoms):            
                #print("{:5s}{:10.6f}{:10.6f}{:10.6f}".format(t1[j], t2[j][0], t2[j][1], t2[j][2]))
                file.write("{:5s}{:10.6f}{:10.6f}{:10.6f}\n".format(t1[j], t2[j][0], t2[j][1], t2[j][2]))
    file.close()
    return

def write_gjf(fn, numatoms, nummols, molecules):
    file = open(fn + '.gjf', 'w')
    file.write("# RHF/6-31G*\n\nTest of fragment output for threshold {:3.1f}\n\n0 1\n".format(distance_threshold))
    for i in molecules:
        if len(i.get_elements()) == numatoms:
            t1 = i.get_elements()
            t2 = i.get_coords()
            for j in range(numatoms):            
                #print("{:5s}{:10.6f}{:10.6f}{:10.6f}".format(t1[j], t2[j][0], t2[j][1], t2[j][2]))
                file.write("{:5s}{:10.6f}{:10.6f}{:10.6f}\n".format(t1[j], t2[j][0], t2[j][1], t2[j][2]))
    file.write('\n')
    file.close()    

In [12]:
write_xyz('test', full_molecule_numatoms, num_full_molecules, molecules)

In [11]:
atom_counts = [len(i.get_elements()) for i in molecules]
for i in sorted(list(set(atom_counts))):
   print("{}: {}".format(i, atom_counts.count(i)))

1: 36
2: 24
3: 18
4: 8
7: 4
9: 12
10: 2
12: 2
13: 4
16: 8
22: 2
23: 2
26: 2
32: 2
35: 4
50: 8
58: 10
